In [ ]:
!pip install tensorflow-gpu==1.9.0 Keras==2.1.6

In [ ]:
!sudo apt-get install graphviz -y


In [ ]:
!pip install pydot


In [1]:
from tensorflow.python import keras as tf_keras


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [32]:
import pydot

import numpy as np
#import keras
import tensorflow as tf


from tensorflow.python import keras

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.metrics import categorical_crossentropy
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.models import Model


from tensorflow.python.keras.layers import Dense, Activation
from tensorflow.python.keras.optimizers import Adam, SGD, RMSprop



from sklearn.metrics import confusion_matrix



#from tensorflow.python.keras.applications.vgg16 import VGG16
#from tensorflow.python.keras import models
#from tensorflow.python.keras import layers

import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from tensorflow.python.keras.utils import vis_utils

In [4]:

#from tensorflow.python.keras.applications import imagenet_utils
from tensorflow.python.keras.utils.vis_utils import model_to_dot

In [5]:
from IPython.display import SVG


In [6]:
!ls data

train  valid


In [7]:
train_folder = "data/train"
valid_folder = "data/valid"

In [8]:

IMAGE_SIZE=224
BATCH_SIZE=64
NUM_CLASSES=120

In [9]:
!ls data/train/ | head

affenpinscher
afghan_hound
african_hunting_dog
airedale
american_staffordshire_terrier
appenzeller
australian_terrier
basenji
basset
beagle


In [10]:
#from keras.preprocessing.image import ImageDataGenerator


In [11]:
tf_keras.preprocessing.image.ImageDataGenerator

tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [ ]:
train_generator = tf_keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input
                                     ,rotation_range=45
                                     ,width_shift_range=0.2
                                     ,height_shift_range=0.2
                                     ,shear_range=0.2
                                     ,zoom_range=0.25
                                     ,horizontal_flip=True
                                     ,fill_mode='nearest'
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  tf_keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

In [ ]:
NUM_CLASSES = train_batches.num_classes

In [ ]:
from tensorflow.python.keras.applications.mobilenet import MobileNet


In [ ]:
base_model = MobileNet()

In [ ]:
#base_model.summary()

In [ ]:
x = base_model.layers[-6].output
#predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
#!pip install tensorflow

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
#model.summary()

In [ ]:
#model.layers[:-1]

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    

In [ ]:
#SVG(model_to_dot(model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))


In [ ]:
#model.summary()

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=train_batches.n//BATCH_SIZE, 
                    validation_data=valid_batches
                    ,  validation_steps=valid_batches.n // BATCH_SIZE, verbose=2,workers=4
                    , epochs=3
                   
                   )


In [ ]:
train_batches.class_indices.keys()

In [ ]:
train_batches.class_indices

In [16]:
model_name = 'dog_breed_tf.h5'

In [ ]:
model.save(model_name)

In [ ]:
with open('labels.txt', 'w') as file_handler:
    for item in train_batches.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [ ]:
!pip install tensorflowjs 

In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras dog_breed_tf.h5 model/

In [18]:
custom_objects={'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}

model = tf.keras.models.load_model(model_name,custom_objects=custom_objects )

# Estimator

In [13]:
import tensorflow as tf

In [14]:
model_dir ="tf_models"

In [19]:
custom_objects={'relu6': tf_keras.applications.mobilenet.relu6,'DepthwiseConv2D': tf_keras.applications.mobilenet.DepthwiseConv2D}
dogbreed_estimator = tf.keras.estimator.model_to_estimator(keras_model=model,
                                                    model_dir=model_dir,custom_objects=custom_objects )



INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tf_models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f95fdc69860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
?dogbreed_estimator.export_savedmodel

In [23]:
import os
import tensorflow as tf
import tensorflow.keras.backend as K

In [30]:
def save_model_for_production(model, version, path='prod_models'):
    K.set_learning_phase(0)
    if not os.path.exists(path):
        os.mkdir(path)
    export_path = os.path.join(
        tf.compat.as_bytes(path),tf.compat.as_bytes(version))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    model_input = tf.saved_model.utils.build_tensor_info(model.input)
    model_output = tf.saved_model.utils.build_tensor_info(model.output)

    prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={'inputs': model_input},
            outputs={'output': model_output},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    with K.get_session() as sess:
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                'predict':
                    prediction_signature,
            })

        builder.save()

In [40]:
import shutil

In [41]:
?shutil.rmtree

In [70]:
def export_saved_model_old(model,path="prod_models"):
    shutil.rmtree(path,ignore_errors=True)
    with tf.Session() as sess:

        signature = tf.saved_model.signature_def_utils.predict_signature_def(                                                                        
        inputs={'image': model.input}, outputs={'scores': model.output})                                                                         

        builder = tf.saved_model.builder.SavedModelBuilder(path)                                                                    
        builder.add_meta_graph_and_variables(                                                                                                        
        sess=K.get_session(),                                                                                                                    
        tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
        signature_def_map={                                                                                                                      
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:                                                                
                signature                                                                                                                        
        })                                                                                                                                       
        builder.save()
        
def export_saved_model(model,path="prod_models"):
    shutil.rmtree(path,ignore_errors=True)

    builder = tf.saved_model.builder.SavedModelBuilder(path)
    signature = tf.saved_model.signature_def_utils.predict_signature_def(
      inputs={"input": model.input},
      outputs={"output": model.output})
 
    with K.get_session() as sess:
        builder.add_meta_graph_and_variables(sess=sess,
        tags=[tf.saved_model.tag_constants.SERVING],
        signature_def_map={"predict": signature, "serving_default": signature} )
    
        builder.save()
        
def export_saved_model3(model,path="prod_models"):
    shutil.rmtree(path,ignore_errors=True)

    with tf.Session() as sess:
    #with K.get_session() as sess:

        signature = tf.saved_model.signature_def_utils.predict_signature_def(                                                                        
        inputs={'image': model.input}, outputs={'scores': model.output})                                                                         

        builder = tf.saved_model.builder.SavedModelBuilder(path)                                                                    
        builder.add_meta_graph_and_variables(                                                                                                        
            sess = sess,
            #sess=K.get_session(),                                                                                                                    
            tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
            signature_def_map={                                                                                                                      
                tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:                                                                
                    signature                                                                                                                        
            })                                                                                                                                       
        builder.save()


In [71]:
#save_model_for_production(model=model,version="1")

In [72]:
export_saved_model3(model=model)

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


FailedPreconditionError: Error while reading resource variable conv1/kernel from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/conv1/kernel)
	 [[Node: conv1/kernel/Read/ReadVariableOp = ReadVariableOp[_class=["loc:@conv1/kernel"], dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1/kernel)]]

Caused by op 'conv1/kernel/Read/ReadVariableOp', defined at:
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in

In [59]:
#export_saved_model(model=model)

# Toco 

In [33]:
tf.contrib.lite.TocoConverter.from_frozen_graph('prod_models/1/saved_model.pb')

TypeError: from_frozen_graph() missing 2 required positional arguments: 'input_arrays' and 'output_arrays'

In [20]:

converter = tf.contrib.lite.TocoConverter.from_keras_model_file(model_name)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

AttributeError: type object 'TocoConverter' has no attribute 'from_keras_model_file'

In [ ]:
?dogbreed_estimator.export_savedmodel

In [ ]:
dogbreed_estimator.e

In [ ]:
dogbreed_estimator = tf.keras.estimator.model_to_estimator(keras_model=model,
                                                    model_dir=model_dir)

In [ ]:
tra

In [ ]:
!gsutil cp /home/ubuntu/custom/deeplizrd/dog_breed.h5 gs://np-training-public/

In [ ]:
!pwd

In [ ]:

!ls /home/ubuntu/.kaggle/competitions/dog-breed-identification/train_folder/

